In [83]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelBinarizer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mutual_info_score, mean_absolute_percentage_error, mean_squared_error, r2_score, mean_absolute_error
from sklearn.feature_selection import mutual_info_regression, SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import SGDRegressor, Ridge
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor

In [5]:
data = pd.read_csv('data.csv', delimiter=';')
data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,16,U,LE3,T,4,3,teacher,services,...,5,4,3,1,2,1,2,16,15,15
1,GP,M,18,U,LE3,T,1,1,other,other,...,2,3,5,2,5,4,0,6,5,0
2,GP,M,17,R,LE3,A,4,4,teacher,other,...,3,3,3,2,3,4,2,10,11,12
3,GP,F,15,U,LE3,T,3,2,services,other,...,4,4,4,1,1,5,10,7,6,6
4,GP,M,16,U,GT3,T,2,3,other,other,...,5,3,3,1,1,3,0,13,14,14


In [6]:
X = data.drop(['G1', 'G2', 'G3'], axis =1)
y = data['G3']


In [7]:
#manually binarize bc the column transformer doesn't like label binarizer 
lb = LabelBinarizer()
for x in  ['school', 'sex', 'address', 'famsize', 'Pstatus', 'schoolsup', 'famsup', 'paid','activities','nursery','internet','higher','romantic']:
    X[x] = lb.fit_transform(X[x])

In [8]:
#engineered features: 
X["Pedu"] = (X['Medu'] + X['Fedu'])/2 #parent education avg
X['Talc'] = (X['Dalc'] + X['Walc'])/2 #total alc consumption 
X['social'] = X['goout'] + X['romantic'] + X['famrel'] #sum of goout and romantic and famrel, max value of 11

In [87]:
#OHE, will happen after test/train/split
ohe = OneHotEncoder(drop ='first')
ord = OrdinalEncoder()

col_trans = ColumnTransformer([('onehot', ohe, ['Mjob', 'Fjob', 'reason', 'guardian'])],
                                    remainder = 'passthrough')
ord_trans = ColumnTransformer([('ord', ord, ['Mjob', 'Fjob', 'reason', 'guardian'])],
                                    remainder = 'passthrough')

In [91]:
#split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state= 1)

In [93]:
X_train = pd.DataFrame(ord_trans.fit_transform(X_train),columns = [s.split('__')[1] for s in ord_trans.get_feature_names_out()])
X_test = pd.DataFrame(ord_trans.fit_transform(X_test),columns = [s.split('__')[1] for s in ord_trans.get_feature_names_out()])


In [75]:
#OHC
X_train = pd.DataFrame(col_trans.fit_transform(X_train),columns = [s.split('__')[1] for s in col_trans.get_feature_names_out()])
X_test = pd.DataFrame(col_trans.fit_transform(X_test),columns = [s.split('__')[1] for s in col_trans.get_feature_names_out()])

In [89]:
X_train.head()

,Mjob,Fjob,reason,guardian,school,sex,age,address,famsize,Pstatus,...,famrel,freetime,goout,Dalc,Walc,health,absences,Pedu,Talc,social
0,3.0,3.0,1.0,1.0,0.0,0.0,18.0,1.0,0.0,1.0,...,5.0,3.0,4.0,1.0,1.0,4.0,0.0,3.0,1.0,9.0
1,0.0,2.0,0.0,1.0,1.0,0.0,18.0,0.0,1.0,0.0,...,4.0,3.0,4.0,1.0,4.0,5.0,0.0,2.5,2.5,9.0
2,0.0,2.0,3.0,1.0,0.0,1.0,16.0,1.0,0.0,1.0,...,5.0,3.0,3.0,1.0,3.0,2.0,10.0,2.5,2.0,9.0
3,3.0,4.0,1.0,1.0,0.0,0.0,17.0,1.0,0.0,1.0,...,4.0,2.0,4.0,2.0,3.0,2.0,24.0,4.0,2.5,8.0
4,0.0,3.0,0.0,0.0,1.0,0.0,18.0,1.0,0.0,1.0,...,5.0,2.0,3.0,1.0,2.0,4.0,0.0,2.5,1.5,9.0


In [32]:
# finding best feature selection for a given model
def test_features(model, selectors): 
    stats = pd.DataFrame(columns = ['selector', 'features', 'r2', 'mse', 'mae', 'relative error'])
    for selector in selectors:
        selector.fit(X_train, y_train)
        features = list(selector.get_feature_names_out())
        train_temp = X_train[features]
        test_temp = X_test[features]
        model.fit(train_temp, y_train)
        pred = model.predict(test_temp)
        r2 = r2_score(y_test, pred)
        mse = mean_squared_error(y_test, pred)
        #a proxy for percent error- need to fix
        mae = mean_absolute_error(y_test, pred)
        relative_e = mae/y_test.mean()
        stats.loc[len(stats.index)]= [str(selector), features, r2, mse, mae, relative_e]
    return(stats)


In [ ]:
test_features()

In [109]:
#get the best feature combination for each model type 
def compare_models(models, selectors, metric):
    stats = pd.DataFrame(columns = ['model', 'k features', 'features', 'r2', 'mse', 'mae', 'relative error'])
    for m in models:
        temp = test_features(m, selectors)
        if metric == 'r2': 
            best = [str(m)] + temp[temp[metric] == temp[metric].max()].values.tolist()[0]
        else: best = [str(m)] + temp[temp[metric] == temp[metric].min()].values.tolist()[0]
        best[1] = len(best[2])
        stats.loc[len(stats.index)] = best
    stats = stats.sort_values(by = metric).reset_index(drop=True)
    return stats 


In [34]:
#tune model
def tune_model(model, x, params):
    tuner = GridSearchCV(estimator=model, param_grid = params, scoring= 'r2')
    tuner.fit(x,y_train)
    return pd.DataFrame(tuner.cv_results_).sort_values(by='rank_test_score')[['params', 'mean_test_score']]

In [102]:
#models and selectors 
models = [SVR(), SGDRegressor(), RandomForestRegressor(), Ridge(), KNeighborsRegressor(), GradientBoostingRegressor()]
selectors = [SelectKBest(k = i, score_func = mutual_info_regression) for i in range (3, len(X_train.columns)+1)]

In [112]:
params = {"SVR()": {'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'], #SVR
          'gamma' : [10**x/10000 for x in range (3)],
          'C' : [10**x/100 for x in range (0,3)]},
          "SGDRegressor()": {'loss' : ['squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],   #SGD
           'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive']   
          },
          "RandomForestRegressor()": {'n_estimators' : range(10, 160, 10),     #Random Forest
           'max_features' : ['sqrt', 'log2', None]
          },
          "Ridge()": {'alpha' : [10**x/100000 for x in range(0, 10,2)]  #Ridge
          },
          "KNeighborsRegressor()": {'n_neighbors' : range(2,21),     #KNeighbors
           'weights'  : ['uniform', 'distance']     
          },
          "GradientBoostingRegressor()": {
            'loss' : ['squared_error', 'absolute_error', 'huber', 'quantile'],  #Gradient Boosting
            'n_estimators' : range(10, 160, 10)
          }
}

In [110]:
best_features = compare_models(models, selectors, 'relative error')


In [99]:
best_features.head(6)

,model,k features,features,r2,mse,mae,relative error
0,SVR(),33,"[Mjob, Fjob, reason, guardian, school, sex, ag...",0.047491,20.262838,3.348735,0.301545
1,SGDRegressor(),16,"[Mjob, Fjob, address, Medu, studytime, failure...",0.087712,19.407213,3.303716,0.297491
2,RandomForestRegressor(),18,"[Mjob, guardian, famsize, Fedu, traveltime, st...",0.255952,15.828223,3.074842,0.276882
3,Ridge(),5,"[Mjob, Medu, failures, romantic, absences]",0.134554,18.410742,3.268851,0.294351
4,KNeighborsRegressor(),20,"[Mjob, reason, sex, age, address, Medu, Fedu, ...",0.215596,16.686737,3.050526,0.274692
5,GradientBoostingRegressor(),6,"[Mjob, guardian, failures, Walc, absences, soc...",0.343061,13.975159,3.006631,0.270739


In [114]:
#tune hyperparameters for top 3 models
stats = {}
for i in (best_features.index[:3]): 
    model = eval(best_features.iloc[i]['model'])
    features = best_features.iloc[i]['features']
    hps = params[best_features.iloc[i]['model']]
    results = tune_model(model, X_train[features], hps)
    print(f"doing {model}")
    stats[(str(model), str(features))] = results

doing RandomForestRegressor()
doing GradientBoostingRegressor()
doing KNeighborsRegressor()


In [115]:
#apply tuned hyperparameters 
tuned_stats = pd.DataFrame(columns = ['model', 'params','n','features', 'r2', 'mse', 'mae', 'relative error'])
for k,v in stats.items():
    params = v['params'][0]
    features = eval(k[1])
    model = eval(k[0][:-2])(**params)
    model.fit(X_train[features], y_train)
    pred = model.predict(X_test[features])
    r2 = r2_score(y_test, pred)
    mse = mean_squared_error(y_test, pred)
    mae = mean_absolute_error(y_test, pred)
    relative_e = mae/y_test.mean()
    tuned_stats.loc[len(tuned_stats.index)] = [k[0], params, len(features), features, r2, mse, mae, relative_e]

In [116]:
tuned_stats.head(6)

,model,params,n,features,r2,mse,mae,relative error
0,RandomForestRegressor(),"{'max_features': 'sqrt', 'n_estimators': 10}",18,"[Mjob, address, Fedu, traveltime, failures, sc...",0.151076,18.059263,3.230526,0.290900
1,GradientBoostingRegressor(),"{'loss': 'squared_error', 'n_estimators': 10}",4,"[Mjob, failures, health, absences]",0.162224,17.822110,3.279305,0.295293
2,KNeighborsRegressor(),"{'n_neighbors': 2, 'weights': 'uniform'}",21,"[Mjob, guardian, sex, age, address, famsize, F...",-0.057549,22.497368,3.384211,0.304739
